## Design Pattern


### SOLID 원칙

객체지향 설계 5대 원칙 (SRP, OCP, LSP, ISP, DIP)을 의미한다.


#### 1. Single Responsibility Principle

단일 책임 원칙, 한 클래스는 단 하나의 책임만 가진다. 만약 여러 책임이 합쳐지면, 변경이나 유지보수가 어려워진다.

In [ ]:
class DataManager:
    def read_data(self, filename):
        # 파일에서 데이터 읽기
        pass

    def process_data(self, data):
        # 데이터 처리
        pass

    def save_report(self, data, out_file):
        # 처리 결과를 보고서 형태로 저장
        pass

위 클래스의 경우, 하나의 클래스 내에서 읽기와 프로세싱, 파일 저장까지 모두 이뤄지고 있다. 하나의 클래스에 여러개의 책임이 따르면, 직관성이 떨어지고 유지보수가 복잡해진다.

In [ ]:
class DataReader:
    '''
    데이터만 읽는 클래스
    '''
    def read_data(self, filename):
        # 파일에서 데이터만 읽는다
        pass

class DataProcessor:
    """
    데이터 처리만 하는 클래스
    """
    def process_data(self, data):
        # 데이터 처리만 한다
        pass

class ReportSaver:
    """
    데이터 분석 결과를 저장하는 클래스
    """
    def save_report(self, data, out_file):
        # 결과를 저장만 한다
        pass

위 예시처럼 단일 책임으로 분리한다. 각각의 변경(입출력, 처리, 저장) 요구사항이 클래스별로 독립된다.

#### 2. OCP (Open/Closed Principle)

개방-폐쇄 원칙, 확장에는 열려 있고, 변경에는 닫혀 있어야 한다. 기존 코드를 수정하지 않고, 새로운 기능이나 구현을 추가해도 확장이 가능하도록 설계한다.

In [1]:
# 좋지 않은 예
class Slippage :
    def calculate_commission(self, trade_type, amount):
        if trade_type == "stock":
            return amount * 0.001
        elif trade_type == "option":
            return amount * 0.002

위 예시는 `trade_type`가 늘어날 때마다 `if/elif`를 수정해야 하므로 변경에 닫혀 있지 않다. 추상 클래스를 사용해 클래스를 인터페이스화 한다면 확장에 더욱 유연해진다.

In [2]:
from abc import ABC, abstractmethod

class CommissionStrategy(ABC): # 추상 클래스 
    @abstractmethod
    def calculate(self, amount): # abstract method를 이용해 상속받을 클래스가 필시 포함되어야 하는 method를 정의한다.
        pass

class StockCommission(CommissionStrategy):
    '''
    stock commission을 계산하는 단일 클래스
    '''
    def calculate(self, amount):
        return amount * 0.0015

class OptionCommission(CommissionStrategy):
    '''
    option commision을 계산하는 단일 클래스
    '''
    def calculate(self, amount):
        return amount * 0.0025
    
class ShortSellingCommission(CommissionStrategy):
    """
    주식 차입 수수료를 계산하는 단일 클래스
    """
    def calculate(self, amount):
        return amount * 0.03

class CommissionCalculator:
    def __init__(self, strategy: CommissionStrategy):
        self.strategy = strategy # 클래스 자제를 인스턴스 변수화

    def calculate_commission(self, amount):
        return self.strategy.calculate(amount)

In [4]:
calculator = CommissionCalculator(StockCommission()) # class 자체를 입력으로 넣는다.
fee = calculator.calculate_commission(100000)

In [5]:
fee

150.0

새로운 대상을 추가할 때에는 클래스만 추가하면 되며, 기존 클래스 코드는 수정하지 않고도 확장된다.

#### 3. LSP (Liskov Substitution Principle)

리스코프 치환 원칙, 자식 클래스는 부모 클래스로 대체(치환)될 수 있어야 한다. 즉, 부모 클래스(인터페이스)가 기대하는 동작이나 규약을 자식 클래스도 위배하지 않아야 한다.

In [6]:
class Asset:
    def get_price(self) -> int:
        pass

class Stock(Asset):
    def get_price(self) -> int:
        return 100  # 정상적인 숫자 반환

class Bond(Asset):
    def get_price(self) -> None:
        return None  # 규약에 어긋나는 반환 (가격 불가)

def print_asset_price(asset: Asset):
    price = asset.get_price()
    print(f"Price is {price}")

In [7]:
bond = Bond()
print_asset_price(bond)  # None 이 반환되어 로직상 에러가 날 수 있음

Price is None


위 예시는 `Bond`가 부모(Asset)의 기대 동작(가격을 숫자로 반환)을 충족하지 않아, 부모 타입으로서 치환이 깨진다.

In [9]:
class Asset:
    def get_price(self) -> int:
        pass

class Stock(Asset):
    def get_price(self) -> int:
        return 100

class Bond(Asset):
    def get_price(self):
        return 90  # 부모와 동일하게 숫자 반환

def print_asset_price(asset: Asset):
    price = asset.get_price()
    print(f"Price is {price}")

In [10]:
bond = Bond()
print_asset_price(bond)  # 정상적으로 90 출력

Price is 90


#### 4. ISP (Interface Segregation Principle)

인터페이스 분리 원칙, 덩치 큰 인터페이스 하나를 만들지 말고, 작고 구체적인 인터페이스 여러 개로 분리한다. 불필요한 메서드를 구현할 의무가 생기지 않도록 설계한다.

In [11]:
class TradingOperations:
    def buy_stock(self):
        pass
    def sell_stock(self):
        pass
    def write_option(self):
        pass
    def exercise_option(self):
        pass
    # 모든 트레이딩 관련 메서드를 하나에 다 넣었다

class StockTrader(TradingOperations):
    def buy_stock(self):
        pass
    def sell_stock(self):
        pass
    def write_option(self):
        pass  # 사용하지 않는 기능도 구현해야 함
    def exercise_option(self):
        pass  # 사용하지 않는 기능도 구현해야 함

위 경우, `StockTrader`는 옵션 관련 메서드를 전혀 쓸 일이 없지만, 인터페이스에 포함된 탓에 구현해야 한다.

In [12]:
from abc import ABC, abstractmethod

class StockTrading(ABC):
    @abstractmethod
    def buy_stock(self):
        pass
    @abstractmethod
    def sell_stock(self):
        pass

class OptionTrading(ABC):
    @abstractmethod
    def write_option(self):
        pass
    @abstractmethod
    def exercise_option(self):
        pass

class StockTrader(StockTrading):
    def buy_stock(self):
        pass
    def sell_stock(self):
        pass

class OptionTrader(OptionTrading):
    def write_option(self):
        pass
    def exercise_option(self):
        pass

위 코드처럼 필요한 인터페이스만 구현하도록 분리하면 필수 기능만을 공유하도록 설계할 수 있다.

#### 5. DIP (Dependency Inversion Principle)

의존성 역전 원칙, 상위(고수준) 모듈이 하위(저수준) 모듈 세부 구현에 의존하지 않도록 하고, 추상화(인터페이스)에 의존하게 만든다.

In [13]:
class FileLogger:
    def write_log(self, msg):
        with open("log.txt", "a") as f:
            f.write(msg + "\n")

class TradingSystem:
    def __init__(self):
        self.logger = FileLogger()  # 구체 클래스 직접 의존

    def trade(self, info):
        # 트레이딩 로직
        self.logger.write_log("Trade executed: " + info)

위 경우 `TradingSystem`은 `FileLogger`라는 구현에 직접 의존한다. 만약 데이터베이스에 로그를 쓰도록 바꾸려면 `TradingSystem` 코드를 수정해야 한다.

In [14]:
from abc import ABC, abstractmethod

class Logger(ABC):
    @abstractmethod
    def write_log(self, msg):
        pass

class FileLogger(Logger):
    def write_log(self, msg):
        with open("log.txt", "a") as f:
            f.write(msg + "\n")

class DbLogger(Logger):
    def write_log(self, msg):
        # DB에 로그 저장
        pass

class TradingSystem:
    def __init__(self, logger: Logger):
        self.logger = logger  # 추상화(인터페이스)에 의존

    def trade(self, info):
        # 트레이딩 로직
        self.logger.write_log("Trade executed: " + info)

In [15]:
ts = TradingSystem(FileLogger())
ts.trade("AAPL BUY 100")

위 예시에서는 `TradingSystem`이 Logger 인터페이스에 의존한다. 구체 로거를 교체해도 상위 모듈 코드를 수정할 필요가 없다.

### Design Pattern

OOP에서 자주 사용되는 해결책 패턴을 의미한다. 자주 발생하는 설계 문제를 효율적으로 해결하고, 가독성과 유지보수 및 원활한 협업을 목표로 한다.

#### 1. Singleton

- 애플리케이션에서 특정 클래스의 인스턴스가 단 하나만 존재하도록 보장한다.
- 인스턴스가 여러 개 생기면 안 되거나, 전역적으로 공유되는 리소스(예: 설정값, 로거) 등에 주로 사용한다.
- 어디서든 동일한 인스턴스에 접근 가능하다. (전역 접근)

In [16]:
class Singleton:
    _instance = None  # 클래스 변수로 인스턴스 저장

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

**`__new__` method를 사용하는 경우**

- `__new__` method를 오버라이드하여 객체 생성 전에 기존 인스턴스를 반환하도록 설정.
- 최초 호출 시 객체를 생성하고, 이후 호출에서는 같은 객체를 반환.

In [17]:
singleton1 = Singleton()
singleton2 = Singleton() # instance

In [18]:
singleton1 is singleton2 # 동일한 인스턴스로 인식함

True

**Decorator로 구현하는 경우**

- `@singleton` 데코레이터를 통해 Singleton 패턴을 적용.
- 클래스 인스턴스를 생성할 때 기존 객체가 없으면 생성하고, 있으면 반환.

In [19]:
def singleton(cls):
    instances = {}  # 인스턴스를 저장할 딕셔너리

    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]

    return get_instance

@singleton
class PortfolioManager:
    def __init__(self):
        self.assets = []

    def add_asset(self, asset):
        self.assets.append(asset)

In [20]:
manager1 = PortfolioManager()
manager2 = PortfolioManager()

In [22]:
manager1.add_asset("AAPL") # manager 1에 AAPL 지정

In [24]:
manager1.assets

['AAPL', 'AAPL']

In [25]:
manager2.assets # 동일한 assets를 공유

['AAPL', 'AAPL']

**Meta class를 사용하는 경우**

- 메타클래스를 활용하여 Singleton을 적용.
- `__call__`을 override하여 인스턴스를 단 하나만 생성.

In [26]:
class SingletonMeta(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class TradingStrategy(metaclass=SingletonMeta):
    def __init__(self, strategy_name):
        self.strategy_name = strategy_name

In [27]:
strategy1 = TradingStrategy("Mean Reversion")
strategy2 = TradingStrategy("Momentum") # 저장되지 않는다

In [28]:
strategy1.strategy_name

'Mean Reversion'

In [29]:
strategy2.strategy_name

'Mean Reversion'

#### 2. Factory Method

- 객체 생성 로직을 서브클래스에 위임하는 방식이다.
- 상위 클래스(또는 인터페이스)는 객체를 생성하는 **‘팩토리 메서드’** 를 갖고, 실제 구현은 자식 클래스가 결정한다.
- 새로운 구체 클래스가 추가되어도 상위 클래스 코드를 수정하지 않고 확장할 수 있다.

In [30]:
from abc import ABC, abstractmethod

# 1. Product Interface (Abstract Class)
class FinancialInstrument(ABC):
    """
    금융 상품의 인터페이스를 먼저 정의한다
    """
    @abstractmethod
    def get_details(self):
        pass

In [31]:
# 2. Concrete Products (Specific Financial Instruments)
# 각각의 금융 상품을 따로 정의한다.
class Stock(FinancialInstrument):
    def get_details(self):
        return "This is a Stock."

class Bond(FinancialInstrument):
    def get_details(self):
        return "This is a Bond."

class Option(FinancialInstrument):
    def get_details(self):
        return "This is an Option."

In [32]:
# 3. Creator (Factory Method Interface)
class FinancialInstrumentFactory(ABC):
    """
    factory method 정의
    """
    @abstractmethod
    def create_instrument(self):
        pass

In [33]:
# 4. Concrete Creators (Specific Factories)
# 각 금융 상품별 객체 생성
class StockFactory(FinancialInstrumentFactory):
    def create_instrument(self):
        return Stock()

class BondFactory(FinancialInstrumentFactory):
    def create_instrument(self):
        return Bond()

class OptionFactory(FinancialInstrumentFactory):
    def create_instrument(self):
        return Option()

In [34]:
# 5. Client Code
# 팩토리 메서드를 호출하여 금융 상품 생성
def get_instrument(factory: FinancialInstrumentFactory):
    instrument = factory.create_instrument()
    print(instrument.get_details())

In [35]:
get_instrument(StockFactory())

This is a Stock.


In [36]:
get_instrument(BondFactory())

This is a Bond.


In [37]:
get_instrument(OptionFactory())

This is an Option.


- 객체 자체를 동적으로 생성할 수 있음
- 거래 전략 등을 캡슐화할 수 있음
- 데이터 소스를 팩토리로 분리해 관리할 수 있음


#### 3. Strategy

- 알고리즘(또는 행위)을 캡슐화하여 런타임에 교체 가능하도록 만든다. 예를 들어, 로직을 여러 가지 전략으로 구분하고, 상황에 따라 다른 전략 객체를 주입해 사용한다.
- 트레이딩 전략, 포트폴리오 리밸런싱 전략, 옵션 가격 결정 전략 등을 유연하게 적용할 때 유용하다
    - 즉, 전략 자체를 개별적인 클래스로 생성하고 이를 추상화하는 인터페이스와 context class로 나누어 관리한다.

In [38]:
from abc import ABC, abstractmethod

# 1. Strategy Interface (Abstract Class)
class TradingStrategy(ABC):
    @abstractmethod
    def execute(self, prices):
        pass

In [39]:
# 2. Concrete Strategies
class MomentumStrategy(TradingStrategy):
    def execute(self, prices):
        if prices[-1] > prices[-5]:  # 최근 가격이 과거보다 상승했으면 매수
            return "Buy (Momentum Strategy)"
        return "Sell (Momentum Strategy)"

class MeanReversionStrategy(TradingStrategy):
    def execute(self, prices):
        avg_price = sum(prices) / len(prices)
        if prices[-1] < avg_price:  # 최근 가격이 평균보다 낮으면 매수
            return "Buy (Mean Reversion Strategy)"
        return "Sell (Mean Reversion Strategy)"

In [40]:
# 3. Context Class (to select strategy)
class TradingContext:
    '''
    현재 상태에서 어떤 전략을 선택하는지 context를 지정
    '''
    def __init__(self, strategy: TradingStrategy):
        self.strategy = strategy

    def set_strategy(self, strategy: TradingStrategy):
        self.strategy = strategy

    def execute_strategy(self, prices):
        return self.strategy.execute(prices)

In [41]:
import yfinance as yf

data = yf.download(
    'AAPL',
    start = '2020-01-01',
    progress = False,
    interval = '1d'
)

In [42]:
prices = data['Close']

In [45]:
context = TradingContext(MomentumStrategy()) # user context 지정
context.execute_strategy(prices.tolist())

'Sell (Momentum Strategy)'

In [46]:
context.set_strategy(MeanReversionStrategy())
context.execute_strategy(prices.tolist())

'Sell (Mean Reversion Strategy)'

#### 4. Observer

- **주체(Subject)** 와 이를 관찰하는 옵저버(Observer) 간 일 대 다 의존 관계를 설정한다.
- 주체의 상태가 변하면, 연결된 옵저버들에게 자동으로 알림을 보낸다.
- 이벤트 기반 시스템, GUI, 금융 시세 업데이트, 리스크 관리 시스템, 실시간 가격 모니터링 등에서 흔히 사용한다.

In [55]:
# Subject (Publisher)
class StockPricePublisher:
    def __init__(self, ticker):
        self._observers = []
        self.ticker = ticker
        self.last_price = None

    def register_observer(self, observer):
        self._observers.append(observer)

    def remove_observer(self, observer):
        self._observers.remove(observer)

    def notify_observers(self):
        for observer in self._observers:
            observer.update(self.ticker, self.last_price)

    def fetch_price(self):
        stock = yf.Ticker(self.ticker)
        data = stock.history(period="1min")
        latest_price = data['Close'].iloc[-1]  # Get the latest closing price
        
        if self.last_price != latest_price:
            self.last_price = latest_price
            self.notify_observers()

In [56]:
# Observer Interface
class Observer(ABC):
    @abstractmethod
    def update(self, ticker, price):
        pass

In [57]:
class TradingSystem(Observer):
    def update(self, ticker, price):
        print(f"Trading System - New price for {ticker}: ${price:.2f}")

In [58]:
class PortfolioManager(Observer):
    def update(self, ticker, price):
        print(f"Portfolio Manager - Adjusting portfolio for {ticker} at ${price:.2f}")

In [59]:
class RiskManagement(Observer):
    def update(self, ticker, price):
        print(f"Risk Management - Analyzing risk for {ticker} at ${price:.2f}")

In [60]:
ticker_symbol = "AAPL"  # Apple stock as an example
stock_publisher = StockPricePublisher(ticker_symbol)

# Create observers
trading_system = TradingSystem()
portfolio_manager = PortfolioManager()
risk_management = RiskManagement()

In [61]:
import time
# Register observers
stock_publisher.register_observer(trading_system)
stock_publisher.register_observer(portfolio_manager)
stock_publisher.register_observer(risk_management)

print(f"Monitoring stock price for {ticker_symbol}...")

# Simulate real-time data updates, 가격이 변할 때마다 업데이트 한다.
for _ in range(3):  # Simulate 3 price checks
    stock_publisher.fetch_price()
    time.sleep(5)  # Wait for 5 seconds before checking price again

Monitoring stock price for AAPL...
Trading System - New price for AAPL: $223.22
Portfolio Manager - Adjusting portfolio for AAPL at $223.22
Risk Management - Analyzing risk for AAPL at $223.22
Trading System - New price for AAPL: $223.26
Portfolio Manager - Adjusting portfolio for AAPL at $223.26
Risk Management - Analyzing risk for AAPL at $223.26


#### 5. Decorator

- 기존 객체에 새로운 기능을 동적으로 추가하는 방법이다.
- 상속 대신 **랩핑(Wrapping)** 을 통해 객체에 책임을 덧붙일 수 있다.
- 트랜잭션 로깅, 성능 모니터링, 리스크 관리 등의 기능을 추가할 때 유용하다.

#### Example 5-1. 함수 기반의 decorator pattern

In [62]:
def transaction_logger(func):
    def wrapper(*args, **kwargs):
        print(f"Executing {func.__name__} with arguments {args}, {kwargs}")
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Finished {func.__name__} in {end_time - start_time:.4f} seconds")
        return result
    return wrapper

In [ ]:
def calculation(principle, rate, years, verbose = True) :
    if verbose : print(f"Calculating {principle} for {years} years")

In [63]:
@transaction_logger
def calculate_compound_interest(principal, rate, years):
    return principal * (1 + rate/100) ** years

decorator의 wrapper가 function call을 할 때마다 실행된다.

In [64]:
print(calculate_compound_interest(1000, 5, 10))

Executing calculate_compound_interest with arguments (1000, 5, 10), {}
Finished calculate_compound_interest in 0.0000 seconds
1628.894626777442


#### Example 5-2. Class 기반의 decorator pattern

In [65]:
# Base financial operation class
class Portfolio:
    def get_return(self):
        return "Calculating portfolio return"

# Decorator class for adding risk management
class RiskManagementDecorator:
    def __init__(self, portfolio):
        self.portfolio = portfolio

    def get_return(self):
        result = self.portfolio.get_return()
        return f"{result} | Risk assessment applied"

In [66]:
portfolio = Portfolio()
secure_portfolio = RiskManagementDecorator(portfolio)

In [68]:
print(secure_portfolio.get_return())

Calculating portfolio return | Risk assessment applied


#### Example 5-3. 다중 데코레이터 적용 (로깅 + 실행 시간 측정)

In [69]:
def log_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"Calling function {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

def time_decorator(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

In [70]:
@log_decorator # 실시간 로그 출력
@time_decorator # 실행 시간 측정
def process_financial_data(amount):
    time.sleep(1)
    return f"Processed {amount} USD"

print(process_financial_data(10000))

Calling function wrapper
process_financial_data took 1.0051 seconds
Processed 10000 USD
